In [1]:
import os 
%pwd

'd:\\Projects\\Machine-Learning-Project-with-MLflow-for-Wine-Quality-Prediction\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Projects\\Machine-Learning-Project-with-MLflow-for-Wine-Quality-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path 
    train_data_path : Path
    test_data_path : Path
    model_name : str 
    alpha : float
    l1_ratio : float
    target_column : str

In [5]:
from wineQuality.constants import *
from wineQuality.utils.common import read_yaml , get_data_from_mongodb ,create_directories

class ConfigManager:
    def __init__(
        self , 
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)


        create_directories([self.config.artifact_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config


In [6]:
import pandas as pd
import os
from wineQuality import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        print(len(train_x))
        print(train_x.shape)
        test_x = test_data.drop([self.config.target_column], axis=1)
        print(len(test_x))
        print(test_x.shape)


        train_y = train_data[[self.config.target_column]]
        print(len(train_y))
        print(train_y.shape)
        test_y = test_data[[self.config.target_column]]
        print(len(train_x))
        print(train_y.shape)


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [11]:
try:
    config = ConfigManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-07-29 23:25:42,764: INFO: common: yaml file: D:\Projects\Machine-Learning-Project-with-MLflow-for-Wine-Quality-Prediction\config\config.yaml loaded successfully]
[2023-07-29 23:25:42,769: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-29 23:25:42,780: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-07-29 23:25:42,783: INFO: common: creating directory at : artifacts]
[2023-07-29 23:25:42,785: INFO: common: creating directory at : artifacts/model_trainer]
1199
(1199, 12)
400
(400, 12)
1199
(1199, 1)
1199
(1199, 1)


In [20]:

csv = pd.read_csv(r'D:\Projects\Machine-Learning-Project-with-MLflow-for-Wine-Quality-Prediction\artifacts\data_ingestion\data.csv', index_col=0)

In [21]:
csv

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
